In [3]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import BertTokenizer
import csv
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel
import time
import datetime
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup
import sys
import argparse
import os
from collections import defaultdict
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from typing import List, Dict
from torch.utils.data import Dataset
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from typing import List
import tqdm
import csv
import json
import faiss  
import pickle

In [4]:
df = pd.read_csv("results.covid_dataset.all.tsv", sep = "\t")
ids, sents = df["sentence_id"].tolist(), df["sentence_text"].tolist()
index = faiss.read_index("output.mean-cls.index")
with open("output.mean-cls.pca.pickle", "rb") as f:
    pca = pickle.load(f)

In [15]:
words = [" spread", " spreading", " spreads", " spreaded"]
relevant_sents_idx = np.array([i for i,sent in enumerate(sents) if len([w for w in words if w in sent]) > 0])[:1000]
id2ind = {ids[i]:i for i,s in enumerate(sents)}
ind2id = {i:ids[i] for i,s in enumerate(sents)}
relevant_sents_ids = np.array([ind2id[i] for i,sent in enumerate(sents) if len([w for w in words if w in sent]) > 0])[:1000]


In [17]:
print([x for x in relevant_sents_ids][:10])

[39, 41, 49, 50, 112, 116, 119, 229, 286, 747]


In [18]:
relevant_vecs = np.array([index.reconstruct(ind.item()) for ind in relevant_sents_idx])
#relevant_vecs = np.array([index.reconstruct(ind) for ind in range(5000)])

In [19]:
mean_relevant = np.mean(relevant_vecs, axis = 0)

In [20]:
D,I = index.search(np.ascontiguousarray([mean_relevant]), 1000)

In [21]:
for i in I.squeeze():
    print(sents[i])
    print("===========================================================")

Despite continued community transmission of SARS in Singapore ( the last community case was identified on May 10 , 2003 ) , the measures implemented since April 8 , 2003 , enabled us to identify and contain eight additional SARS cases and prevent the nosocomial transmission of the virus .
This feature should be especially useful for incorporating known transmission events or introductions of the pathogen into the population , based for instance on clinical investigations and contact tracing information .
One can observe that the effects of including more stages in the disease model on intermediate behaviour are similar to those seen earlier , namely that gamma distribution of infectious period shortens the overall duration of an epidemic and increases peak prevalence .
Another possible explanation could be that underlying passive or active immunity enabled transmission to occur at a rather low rate and that transmission remained continuous in and thus perpetuated the infection within t

In [33]:
sents[:1000]

['Sequences in bold indicate codons requiring multiple nucleotide substitutions .',
 'Poisson regression with analytic weights , offsets , and robust variance estimation was used to implement the above extrapolation and standardization procedures for estimating seasonal incidence and 95 % confidence intervals ( CIs ) [ 17 ] [ 18 ] [ 19 ] .',
 'The analysis data set comprised enrolled patients .',
 'Enrolled patients that did not receive an ICD-9 code for ARI were assigned analytic weights of 1.0 .',
 'Otherwise , analytic weights equal to the inverse of the proportion of cohort members with MAARI that were enrolled were assigned within strata defined by age group ( 6 - 11 , 12 - 23 , and 24 - 59 months ) and season .',
 'These analytic weights were then multiplied by the appropriate adjustment factors as described above .',
 'The outcome in the regression models was the dichotomous virus test result ( coded 1 = positive , 0 = negative ) .',
 'The offset terms were stratified by season 

In [12]:
id2ind = {ids[i]:i for i,s in enumerate(sents)}
ind2id = {i:ids[i] for i,s in enumerate(sents)}

In [11]:
id2ind

{0: 0,
 4: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 8,
 12: 9,
 14: 10,
 15: 11,
 16: 12,
 17: 13,
 18: 14,
 21: 15,
 23: 16,
 24: 17,
 25: 18,
 27: 19,
 29: 20,
 30: 21,
 31: 22,
 32: 23,
 33: 24,
 35: 25,
 37: 26,
 39: 27,
 40: 28,
 41: 29,
 42: 30,
 43: 31,
 44: 32,
 45: 33,
 46: 34,
 47: 35,
 48: 36,
 49: 37,
 50: 38,
 52: 39,
 53: 40,
 54: 41,
 56: 42,
 57: 43,
 59: 44,
 60: 45,
 61: 46,
 63: 47,
 65: 48,
 67: 49,
 68: 50,
 70: 51,
 71: 52,
 72: 53,
 74: 54,
 75: 55,
 76: 56,
 77: 57,
 78: 58,
 79: 59,
 81: 60,
 83: 61,
 84: 62,
 85: 63,
 86: 64,
 87: 65,
 89: 66,
 90: 67,
 91: 68,
 92: 69,
 93: 70,
 94: 71,
 96: 72,
 97: 73,
 98: 74,
 99: 75,
 100: 76,
 101: 77,
 102: 78,
 103: 79,
 105: 80,
 106: 81,
 107: 82,
 108: 83,
 109: 84,
 110: 85,
 111: 86,
 112: 87,
 113: 88,
 114: 89,
 115: 90,
 116: 91,
 117: 92,
 118: 93,
 119: 94,
 120: 95,
 121: 96,
 122: 97,
 123: 98,
 124: 99,
 125: 100,
 126: 101,
 128: 102,
 130: 103,
 131: 104,
 132: 105,
 133: 106,
 134: 107,
 135: